In [86]:
#import modules
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import matplotlib.pylab as plt
import seaborn as sns
import datetime
from datetime import date
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [87]:
#access database
URI = "dublinbikeappdb.cxaxe40vwlui.us-east-1.rds.amazonaws.com"
DB = "dbikes1"
name = "chris01"
pw = "Cd4936234"

"""Initate connection"""
engine = create_engine("mysql+mysqlconnector://{}:{}@{}:3306/{}".format(name,pw,URI,DB),echo=False)

In [132]:
#get list of stations
select = """SELECT a.number, a.bike_stands
FROM availability a
GROUP BY a.number"""

In [133]:
#convert db tables to pandas dataframes
dfs = pd.read_sql_query(select, engine)
dfs.sort_values("number",inplace= True)

#drop final row and convert to list
dfs.drop(dfs.tail(1).index,inplace=True)
stations = dfs.number.tolist()

In [134]:
#load modes into juypter
import pickle
models = pickle.load(open('model_accuary.pkl','rb'))

In [135]:
models

{2: [array([ 0.04094683, -0.60643285,  0.08316337, -0.07336316]),
  1.4643449516967983,
  {'rmse': 3.8455080053801267, 'mae': 3.269023595313988}],
 3: [array([ 0.0111492 ,  0.02265872, -0.01434361, -0.13526034]),
  6.772585041951532,
  {'rmse': 4.136000131608015, 'mae': 3.4610020139118762}],
 4: [array([-0.06483443, -0.85582175, -0.01004544, -0.11249981]),
  12.232754793533172,
  {'rmse': 3.8041416803997024, 'mae': 2.9798466464040065}],
 5: [array([-0.02234386, -0.88540123, -0.20437133, -0.69999159]),
  38.50554595655625,
  {'rmse': 8.547397939332658, 'mae': 6.559494126658559}],
 6: [array([ 0.04585376,  0.12005058, -0.04398746, -0.02722341]),
  5.164194934651624,
  {'rmse': 2.7743955126804543, 'mae': 2.1718591737213644}],
 7: [array([ 0.0475445 ,  0.38471554, -0.08234567, -0.46675874]),
  15.373715261257036,
  {'rmse': 4.796439242820795, 'mae': 3.737256697324708}],
 8: [array([-0.06640555, -0.4650564 , -0.03579098,  0.29941676]),
  11.61219165860927,
  {'rmse': 5.4327308435698995, 'ma

In [136]:
#get rmse values
rmse = []
mae = []

for i in stations:
    rmse.append(models[i][2]['rmse'])
    mae.append(models[i][2]['mae'])

In [137]:
#implement rmse and mae as columns to the pandas dataframe
dfRMSE = pd.DataFrame({"rmse":rmse},index=stations)
dfMAE = pd.DataFrame({"mae":mae},index=stations)

In [138]:
#create pandas dataframe
dfs = dfs.set_index("number")
df = pd.concat([dfs, dfRMSE, dfMAE], axis=1)
df

bike_stands      rmse       mae
2             20  3.845508  3.269024
3             20  4.136000  3.461002
4             20  3.804142  2.979847
5             40  8.547398  6.559494
6             20  2.774396  2.171859
..           ...       ...       ...
113           40  3.060186  2.435210
114           40  5.616401  4.647032
115           30  7.003433  5.708453
116           30  4.488603  3.773697
117           40  2.353668  1.900352

[109 rows x 3 columns]

In [143]:
df["rmse%"] = (df["rmse"] / df["bike_stands"])*100
df["mae%"] = (df["mae"] / df["bike_stands"])*100

In [145]:
#summary statistics
features = ["rmse","mae","rmse%","mae%"]
df[features].describe()

rmse         mae       rmse%        mae%
count  109.000000  109.000000  109.000000  109.000000
mean     5.211832    4.231374   16.812549   13.674221
std      1.634456    1.351279    5.240916    4.467554
min      2.353668    1.844173    5.884171    4.750880
25%      3.982823    3.205837   13.021389   10.511339
50%      5.046973    4.017530   16.385979   13.096701
75%      6.237999    5.195304   20.488123   16.345118
max     10.488780    8.842092   30.969158   26.850734